# Mesh formats

NiiVue can display many mesh formats (GIFTI, PLY, OBJ MZ3, etc). This demo will download various examples and display them. This notebook mirrors this [JavaScript web page](https://niivue.com/demos/features/meshes.html).

In [ ]:
import pathlib
import ipywidgets as widgets
from IPython.display import display
import ipyniivue
from ipyniivue import NiiVue, download_dataset

## Download Formats

DATA_FOLDER = pathlib.Path(ipyniivue.__file__).parent / "images"

local_imgs = [
 "mni152_2009.mz3",
 "ColumnMajorOrder.gii",
 "mni_format.obj.gz",
 "simplify_brain.obj",
 "lh.pial",
 "sub-test02_left_hemisphere.srf.gz",
 "tract.SLF1_R.tck",
 "colby.trk",
 "colby.trx",
 "TR_S_R.tt.gz",
 "tract.FAT_R.vtk",
 "water-color.wrl",
 "MolView-sticks-color_38.x3d",
]

files_to_download_local = local_imgs.copy()

download_dataset(
    api_url="https://niivue.com/demos/images/",
    dest_folder=DATA_FOLDER,
    files=files_to_download_local,
)


## Setup NiiVue Instance

nv = NiiVue(
    back_color=(0.9, 0.9, 1.0, 1),
)
nv.opts.show_legend = False

nv.load_meshes([
        {
            "path": "../images/CIT168.mz3",
        },
        {
            "path": "../images/BrainMesh_ICBM152.lh.mz3",
            "rgba255": [222, 164, 164, 255],
        }
    ])

## Store image paths

img_paths = {}

# Formats
for img in local_imgs:
    img_path = DATA_FOLDER / img
    img_paths[img] = img_path

## User interface dropdowns

# Dropdown for Formats
formats_dropdown = widgets.Dropdown(
    options=[(img, img) for img in local_imgs],
    description="Formats:",
    value=None,
)

## Event handlers

def on_format_change(change):
    """Handle selection in Formats Dropdown."""
    selected_img = change["new"]
    if not selected_img:
        return
    img_path = img_paths.get(selected_img)
    if not img_path.exists():
        print(f"Image {img_path} not found.")
        return
    volumeObj = {
        "path": img_path,
    }
    print("Loading format image:", img_path.name)
    nv.load_meshes([volumeObj])

alpha_options = ["Opaque", "Translucent", "Transparent"]

alpha_dropdown = widgets.Dropdown(
    options=alpha_options,
    value="Opaque",
    description="Opacity:",
)


def on_alpha_change(change):
    """Set mesh layer shader."""
    idx = len(nv.meshes) - 1
    value = change["new"]
    if (value == "Opaque"):
        nv.meshes[idx].opacity = 1.0
    if (value == "Translucent"):
        nv.meshes[idx].opacity = 0.2
    if (value == "Transparent"):
        nv.meshes[idx].opacity = 0.0
    nv.draw_scene()

shader_options = nv.mesh_shader_names()

shader_dropdown = widgets.Dropdown(
    options=shader_options,
    value="Phong",  # Default shader
    description="Shader:",
)


def on_shader_change(change):
    """Set mesh layer shader."""
    idx = len(nv.meshes) - 1
    nv.set_mesh_shader(nv.meshes[idx].id, change["new"])



## Observe the dropdowns

formats_dropdown.observe(on_format_change, names="value")
shader_dropdown.observe(on_shader_change, names="value")
alpha_dropdown.observe(on_alpha_change, names="value")

## Display

controls = widgets.HBox([formats_dropdown, shader_dropdown, alpha_dropdown])
display(controls, nv)